In [ ]:
# Required imports
import re

import hszinc
from pyhaystack.client.niagara import Niagara4HaystackSession

from app import config

In [ ]:
# Creating a session
session = Niagara4HaystackSession(
    uri=config.NIAGARA4_SERVER,
    username=config.NIAGARA4_USERNAME,
    password=config.NIAGARA4_PASSWORD,
    http_args=dict(tls_verify=False, debug=True),
    pint=False,
)
session._grid_format = hszinc.MODE_ZINC

In [ ]:
# Fetching Points
points = session.find_entity("point")
point_results = points.result

In [ ]:
# Total points
print(len(point_results))
point_values = list(point_results.values())

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=["P_Id","D_Id","B_Id","P_Tags", "P_CurrentValue", "P_Type", "P_TimeZone", "P_Unit", "B_Address", "B_Area"])

In [ ]:
def get_site(site_name: str):
    """
    return a site by name
    site_name example: S.~31430Broadway.elecMeter
    """
    op = session.get_entity(site_name)
    op.wait()
    site = op.result
    return site

In [ ]:
def get_device_data(point_id: str, point_name: str):
    device_id = point_id.split(f".{point_name}")[0]
    device_dict = dict(D_Id = device_id)
    try:
        device_data = get_site(device_id)
        building_id = device_id.split(f".{device_data.tags.['navName']}")[0]
        building_data = get_site(building_id)
        device_dict.update({
            "B_Id":building_id,
            "B_Address": building_data.tags.get("geoAddr"),
            "B_Area": building_data.tags.get("area")
            })                        
    except Exception as error:
        print("Error")
    return device_dict
                                      

In [ ]:
for point in point_values:
    temp_dict = {
            "P_Id": point._entity_id,
            "P_Tags": point.tags,
            "P_CurrentValue": point.tags.get("curVal"),
            "P_Type": point.tags.get("kind"),
            "P_TimeZone": point.tags.get("tz"),
            "P_Unit": point.tags.get("unit")
        }
        
    temp_dict.update(get_device_data(point._entity_id, point.tags.get("navName")))
    df = df.append(temp_dict, ignore_index = True)

In [ ]:
 df.head()

In [ ]:
df.to_excel("niagara4_data.xlsx", sheet_name="N4Data", index=False)